# **Extract Transform Load (ETL) Lab**


## Objectives

After completing this lab you will be able to:

-   Read CSV and JSON file types.
-   Extract data from the above file types.
-   Transform data.
-   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions


In [ ]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [ ]:
#!python -m wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

## Unzip Files


In [ ]:
#!python -m unzip source.zip

## Set Paths


In [ ]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "all_data/logs/logfile.txt"            # all event logs will be stored in this file
targetfile = "all_data/output/transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [ ]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function


In [ ]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function


In [38]:
""" def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe """


def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    rows = []
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        rows.append({"name": name, "height": height, "weight": weight})
    if rows:
        dataframe = pd.concat([dataframe, pd.DataFrame(rows)], ignore_index=True)
    return dataframe

### Extract Function


In [36]:
def extract(): 
    extracted_data = pd.DataFrame(columns=['name','height','weight'])  # create an empty data frame to hold extracted data
     
    # process all csv files, except the target file
    for csvfile in glob.glob("all_data/source/*.csv"): 
        if csvfile != targetfile:  # check if the file is not the target file
            extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True) 
            print(extracted_data)
    #process all json files
    for jsonfile in glob.glob("all_data/source/*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True)
        print(extracted_data)
    #process all xml files
    for xmlfile in glob.glob("all_data/source/*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True)
        print(extracted_data)
        
    return extracted_data

## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [39]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading


In [40]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## Logging


In [41]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("all_data/logs/logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [42]:
log("ETL Job Started")

In [43]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

    name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30
    name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30
5   alex   65.78  112.99
6   ajay   71.52  136.49
7  alice   69.40  153.03
8   ravi   68.22  142.34
9    joe   67.79  144.30
     name  height  weight
0    alex   65.78  112.99
1    ajay   71.52  136.49
2   alice   69.40  153.03
3    ravi   68.22  142.34
4     joe   67.79  144.30
5    alex   65.78  112.99
6    ajay   71.52  136.49
7   alice   69.40  153.03
8    ravi   68.22  142.34
9     joe   67.79  144.30
10   alex   65.78  112.99
11   ajay   71.52  136.49
12  alice   69.40  153.03
13   ravi   68.22  142.34
14    joe   67.79  144.30
     name  height  weight
0    alex   65.78  112.99
1    ajay   71.52  136.49
2   alice   69.40  153.03
3    ravi   68.22  142.34
4     joe   67.79  144.30
5  

C:\Users\farra\AppData\Local\Temp\ipykernel_33428\1875367657.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)
C:\Users\farra\AppData\Local\Temp\ipykernel_33428\518166823.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame(rows)], ignore_index=True)
C:\Users\farra\AppData\Local\Temp\ipykernel_33428\518166823.py:24: FutureWarning: The behavior of DataFrame c

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [44]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [45]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [46]:
log("ETL Job Ended")